In [1]:
#30/7/24 Creates a K fold KNN model for each file in a folder provided they are Fingerprints

import os
import pandas as pd
import numpy as np
import math
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Fingerprints/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
resultslist = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        for mtry in range(1, 4):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = SVM = svm.SVC(C=1, kernel='poly', gamma='scale',degree = mtry,
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)


            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = SVM = svm.SVC(C=1, kernel='poly', gamma='scale',degree = bestmetrics,
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)

        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    resultslist.append(results[12])

For dataset NR-AR-LBD_Avafp.csv
for fold 1 test set mcc of 0.6949822166446374 valid set mcc of 0.5464696078082152
for fold 2 test set mcc of 0.5379289185083113 valid set mcc of 0.35087798113269913
for fold 3 test set mcc of 0 valid set mcc of 0.3857583749052298
for fold 4 test set mcc of 0.6153084570802658 valid set mcc of 0
for fold 5 test set mcc of 0.6900721388834448 valid set mcc of 0.4268349027294209

validation metrics of:
positives in data 47
negatives in data 49
net accuracy = 0.7291666666666666
mcc = 0.4757475011684601
For dataset SR-ATAD5_Morganfp.csv
for fold 1 test set mcc of 0.5893689584893059 valid set mcc of 0.5262013605584339
for fold 2 test set mcc of 0.5095693779904307 valid set mcc of 0.2720637033616408
for fold 3 test set mcc of 0.5038378351241242 valid set mcc of 0.6690386626313092
for fold 4 test set mcc of 0.26745913993429266 valid set mcc of 0.5335783750799326
for fold 5 test set mcc of 0.2033125151976111 valid set mcc of 0.4930493313022979

validation metrics o

for fold 4 test set mcc of 0.42031984228460867 valid set mcc of 0.4125991944224991
for fold 5 test set mcc of 0.43366953679424136 valid set mcc of 0.2727547171322793

validation metrics of:
positives in data 181
negatives in data 243
net accuracy = 0.660377358490566
mcc = 0.2907492244720892
For dataset Hepatotoxicity_Avafp.csv
for fold 1 test set mcc of 0.4752254355510956 valid set mcc of 0.3949329359165425
for fold 2 test set mcc of 0.4717534273701748 valid set mcc of 0.439051846374471
for fold 3 test set mcc of 0.5366055867419431 valid set mcc of 0.39593726786055594
for fold 4 test set mcc of 0.4205789710514474 valid set mcc of 0.3747167451213267
for fold 5 test set mcc of 0.453368075044312 valid set mcc of 0.3974180958891304

validation metrics of:
positives in data 293
negatives in data 285
net accuracy = 0.7422145328719724
mcc = 0.48558966098283435
For dataset Reproductive_Toxicity_Morganfp.csv
for fold 1 test set mcc of 0 valid set mcc of 0
for fold 2 test set mcc of 0 valid set 

For dataset NR-AR_Morganfp.csv
for fold 1 test set mcc of 0.6002099318605758 valid set mcc of 0.41306140597069063
for fold 2 test set mcc of 0.5712060961522726 valid set mcc of 0.6333004963811236
for fold 3 test set mcc of 0.5066710044155939 valid set mcc of 0.5862903896799836
for fold 4 test set mcc of 0.5205674283520203 valid set mcc of 0.29569922175366314
for fold 5 test set mcc of 0.5391800344720129 valid set mcc of 0

validation metrics of:
positives in data 61
negatives in data 64
net accuracy = 0.728
mcc = 0.5045381731276043
For dataset Cardiotoxicity-5_MACCSfp.csv
for fold 1 test set mcc of 0.37101600418013425 valid set mcc of 0.40793855370879645
for fold 2 test set mcc of 0.35533563681117736 valid set mcc of 0.3941294733253171
for fold 3 test set mcc of 0.5185629788417315 valid set mcc of 0.4161877458790012
for fold 4 test set mcc of 0.3384180632403519 valid set mcc of 0.5571158118105536
for fold 5 test set mcc of 0.34896645552782174 valid set mcc of 0.4581159620539723

valida

For dataset NR-AR_TTorsionfp.csv
for fold 1 test set mcc of 0.5974235104669887 valid set mcc of 0.37504578475079636
for fold 2 test set mcc of 0.6082291325648282 valid set mcc of 0.600245047998781
for fold 3 test set mcc of 0.4090172651337302 valid set mcc of 0.5235175542131809
for fold 4 test set mcc of 0.5537765319092597 valid set mcc of 0.19218555339901838
for fold 5 test set mcc of 0.5819358307200044 valid set mcc of 0

validation metrics of:
positives in data 61
negatives in data 64
net accuracy = 0.728
mcc = 0.5045381731276043
For dataset NR-ER_Avafp.csv
for fold 1 test set mcc of 0.2683949169273233 valid set mcc of 0.21398266546446335
for fold 2 test set mcc of 0.2793045240898603 valid set mcc of 0.31001630357053045
for fold 3 test set mcc of 0.3343999012234284 valid set mcc of 0.2736580053381785
for fold 4 test set mcc of 0.338419883803869 valid set mcc of 0.318186300552335
for fold 5 test set mcc of 0.31282475480231 valid set mcc of 0.43671629488422387

validation metrics of:


for fold 5 test set mcc of 0.6032061392810438 valid set mcc of 0.4757746940525602

validation metrics of:
positives in data 164
negatives in data 106
net accuracy = 0.7555555555555555
mcc = 0.47817527693080214
For dataset Respiratory_Toxicity_TTorsionfp.csv
for fold 1 test set mcc of 0.3945212650247481 valid set mcc of 0.40933549789595935
for fold 2 test set mcc of 0.4683039027554037 valid set mcc of 0.26883934637698426
for fold 3 test set mcc of 0.4683954809812205 valid set mcc of 0.43519413988924455
for fold 4 test set mcc of 0.5423140244613455 valid set mcc of 0.4855426095137778
for fold 5 test set mcc of 0.569833681621598 valid set mcc of 0.33321483357629716

validation metrics of:
positives in data 164
negatives in data 106
net accuracy = 0.7185185185185186
mcc = 0.39455749431102266
For dataset NR-AR-LBD_TTorsionfp.csv
for fold 1 test set mcc of 0.5345820306956206 valid set mcc of 0.48122447405980645
for fold 2 test set mcc of 0.6356048762998656 valid set mcc of 0.5846695269740967

for fold 5 test set mcc of 0.5719213868315782 valid set mcc of 0.49479247787351405

validation metrics of:
positives in data 70
negatives in data 240
net accuracy = 0.867741935483871
mcc = 0.5868310831530859
For dataset SR-MMP_Avafp.csv
for fold 1 test set mcc of 0.6442533144492408 valid set mcc of 0.5069938353868925
for fold 2 test set mcc of 0.6576966872128501 valid set mcc of 0.6773347307441507
for fold 3 test set mcc of 0.59544204852811 valid set mcc of 0.528291612390944
for fold 4 test set mcc of 0.6991448175706009 valid set mcc of 0.5382918665200542
for fold 5 test set mcc of 0.657974271830879 valid set mcc of 0.48413653375253796

validation metrics of:
positives in data 179
negatives in data 243
net accuracy = 0.7748815165876777
mcc = 0.5382015284992395
For dataset Cardiotoxicity-1_TTorsionfp.csv
for fold 1 test set mcc of 0.5379522766092124 valid set mcc of 0.4700734936051405
for fold 2 test set mcc of 0.4865648040937291 valid set mcc of 0.5458104064524364
for fold 3 test set m

for fold 2 test set mcc of 0 valid set mcc of 0
for fold 3 test set mcc of 0 valid set mcc of 0
for fold 4 test set mcc of 0 valid set mcc of 0
for fold 5 test set mcc of 0 valid set mcc of -0.16666666666666666

validation metrics of:
positives in data 8
negatives in data 6
net accuracy = 0.7142857142857143
mcc = 0.4166666666666667
For dataset Cardiotoxicity-30_Avafp.csv
for fold 1 test set mcc of 0.29715281186899645 valid set mcc of 0.37102171651764854
for fold 2 test set mcc of 0.3959076409815214 valid set mcc of 0.37142857142857144
for fold 3 test set mcc of 0.2305907905068564 valid set mcc of 0.40581426455559233
for fold 4 test set mcc of 0.38432209954297986 valid set mcc of 0.2569970264812942
for fold 5 test set mcc of 0.3990461049284579 valid set mcc of 0.24362919231306998

validation metrics of:
positives in data 85
negatives in data 53
net accuracy = 0.6304347826086957
mcc = 0.22733808711723294
For dataset SR-ATAD5_Avafp.csv
for fold 1 test set mcc of 0.6085989058527203 valid s

In [2]:
temp = pd.DataFrame(resultslist, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Model_Outputs/SVM_Fingerprint_Results.csv')